# Workshop task

Imagine now that you are interested in looking at the relationship between two of Seshat's social complexity variables. For example, the relationship between a transport infrastructure variable such as "Road" and a profession variable like "Professional soldier". Your hypothesis could be that the polities with professional soldiers need roads for them to move on, so there should be a strong correlation between the number of polities with soldiers and the number of polities with roads in any given year.

In [1]:
from seshat_api import SeshatAPI, get_frequencies
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
client = SeshatAPI(base_url="https://seshatdata.com/api")
# client = SeshatAPI(base_url="https://seshat-db.com/api")

In [3]:
years = range(500, 1501)

In [4]:
variables = ['road', 'professional_soldier']

In [5]:
example_frequency_df = get_frequencies(client, variables, years)

In [ ]:
example_frequency_df.sample(5)

In [ ]:
plt.figure(figsize=(13, 7))
plt.plot(example_frequency_df.index, example_frequency_df['professional_soldier'], label='Professional Soldiers')
plt.plot(example_frequency_df.index, example_frequency_df['road'], label='Roads')
plt.xlabel('Year')
plt.ylabel('Number of Polities')
plt.title('Number of Polities with Professional Soldiers and Roads Over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(13, 7))
scatter = plt.scatter(
    example_frequency_df['professional_soldier'], 
    example_frequency_df['road'], 
    c=example_frequency_df.index,
    cmap='viridis',
)
plt.xlabel('Number of polities with Professional Soldiers')
plt.ylabel('Number of polities with Roads')
plt.title('Polities recorded as having Professional Soldiers vs Roads: 500CE to 1500CE')
plt.colorbar(scatter, label='Year')
plt.show()